## KNN Model

In [1]:
# ONLY USE THIS BLOCK OF CODE ON NICASIA'S COMPUTER 
import sys
sys.path.append("/anaconda/lib/python2.7/site-packages")
#####

import pandas as pd
import numpy as np
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer
from sklearn import cross_validation
from sklearn import covariance
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor as KNN
import string
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import math
import string
import os
import datetime
from sklearn.decomposition import PCA

/Users/marinaadario/marinaadario/anaconda3/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_data = pd.read_csv("train_imputed.csv")
train_data.head()

,tax_value,property_size,zestimate_amount,bathrooms,tax_year,latitude,bedrooms,year_built,home_size,longitude,...,neighborhood_South Bronx,neighborhood_Sunny Side,neighborhood_Tremont,neighborhood_Tribeca,neighborhood_Upper East Side,neighborhood_Upper West Side,neighborhood_Washington Heights,neighborhood_West Village,neighborhood_Williamsburg,neighborhood_Yorkville
0,2290000.0,2000.0,849427.0,1.0,2015.0,40.724448,1.0,1910.0,400.0,-73.980284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2837000.0,650.0,1225236.0,1.0,2015.0,40.724225,1.0,1876.0,650.0,-73.979720,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1670000.0,2250.0,7615912.0,1.0,2015.0,40.724095,1.0,1950.0,8720.0,-73.979414,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2290000.0,2000.0,4106754.0,1.0,2015.0,40.723925,3.0,1910.0,3374.0,-73.979737,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2290000.0,2000.0,1417455.0,1.0,2015.0,40.723959,2.0,1910.0,3374.0,-73.979819,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
x_train = train_data.drop(['zestimate_amount'],1)
y_train = train_data['zestimate_amount']

In [4]:
def k_fold_knn(x, y, n_neighbors, num_folds):
    
    kfold = cross_validation.KFold(len(x), num_folds, shuffle = True)

    # Iterate over folds
    cv_rsquared = 0
    
    for train, test in kfold:
        x_train_cv = x.iloc[train].values
        y_train_cv = y.iloc[train].values
        x_test_cv = x.iloc[test].values
        y_test_cv = y.iloc[test].values
        
        # Fit linear regression model and evaluate
        knn_model = KNN(n_neighbors = n_neighbors)
        knn_model.fit(x_train_cv, y_train_cv)
        rsquared = knn_model.score(x_test_cv, y_test_cv)
    
        # Cummulative accuracy across folds
        cv_rsquared += rsquared

    # Return average accuracy across folds
    return cv_rsquared * 1.0 / num_folds

#### Cross validation on KNN models with different numbers of neighbors

In [5]:
for i in range(2, 20): 
    rsquared = k_fold_knn (x_train, y_train, i, 3)
    print "R-squared for k=" + str(i) + " neighbors: " + str(rsquared)

R-squared for k=2 neighbors: -0.0483288892132
R-squared for k=3 neighbors: 0.0745687673224
R-squared for k=4 neighbors: 0.108801780821
R-squared for k=5 neighbors: 0.122334093097
R-squared for k=6 neighbors: 0.147339612788
R-squared for k=7 neighbors: 0.180201986087
R-squared for k=8 neighbors: 0.209480684131
R-squared for k=9 neighbors: 0.18909130198
R-squared for k=10 neighbors: 0.210953469382
R-squared for k=11 neighbors: 0.200894660079
R-squared for k=12 neighbors: 0.227654604891
R-squared for k=13 neighbors: 0.227912713413
R-squared for k=14 neighbors: 0.217613632146
R-squared for k=15 neighbors: 0.216312153713
R-squared for k=16 neighbors: 0.222403743744
R-squared for k=17 neighbors: 0.219114055459
R-squared for k=18 neighbors: 0.214025129345
R-squared for k=19 neighbors: 0.227220963595


The number of neighbors that resulted in the highest cross-validated r-squared was 18, so this will be our final model.

#### Final model


In [6]:
final_knn_model = KNN(n_neighbors = 16)
final_knn_model.fit(x_train.values, y_train.values)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=16, p=2,
          weights='uniform')

#### Performance of final model on test data

In [7]:
test_data = pd.read_csv("test_imputed.csv")

In [8]:
x_test = test_data.drop(['zestimate_amount'],1)
y_test = test_data['zestimate_amount']

In [9]:
final_knn_model.score(x_test.values, y_test.values)

0.23795098246463542

### ADDING PCA TO REDUCE FEATURE SPACE:


In [13]:
train_best_features = pd.read_csv("train_best_features.csv")
test_best_features = pd.read_csv("test_best_features.csv")

In [14]:
x_train = train_best_features.drop(['zestimate_amount'],1)
y_train = train_best_features['zestimate_amount']

In [15]:
x_test = test_best_features.drop(['zestimate_amount'],1)
y_test = test_best_features['zestimate_amount']

In [23]:
def k_fold_knn_with_pca(pca_dims, neighbors, num_folds, x, y):
    
    kfold = cross_validation.KFold(len(x), num_folds, shuffle = True)

    # Iterate over folds
    cv_rsquared = 0
    
    for train, test in kfold:
        x_train_cv = x.iloc[train].values
        y_train_cv = y.iloc[train].values
        x_test_cv = x.iloc[test].values
        y_test_cv = y.iloc[test].values
        
        #Project to the data onto axes
        pca = PCA(n_components=pca_dims)
        pca.fit(x_train_cv)
        x_train_reduced = pca.transform(x_train_cv)
        x_test_reduced = pca.transform(x_test_cv)

        
        # Fit linear regression model and evaluate
        knn_model = KNN(n_neighbors = neighbors)
        knn_model.fit(x_train_reduced, y_train_cv)
        rsquared = knn_model.score(x_test_reduced, y_test_cv)
    
        # Cummulative accuracy across folds
        cv_rsquared += rsquared

    # Return average accuracy across folds
    return cv_rsquared * 1.0 / num_folds

In [27]:
pca_dims = range(5,15)
knn_neighbors = range(20,80,2)

best_r2 = 0
best_pca_dim = 0
best_knn_k = 0
for pca_dim in pca_dims:
    for knn_neighbor in knn_neighbors:
        current_r2 = k_fold_knn_with_pca(pca_dim, knn_neighbor, 4, x_train, y_train)
        print "PCA Dimensions: ", pca_dim, "\tKNN # Neighbors: ", knn_neighbor, "\tR^2: ", current_r2
        if current_r2 > best_r2:
            best_r2 = current_r2
            best_pca_dim = pca_dim
            best_knn_k = knn_neighbor
            
print "BEST HYPERPARAMETERS FOR KNN WITH WITH PCA:"
print "PCA Dimensions: ", best_pca_dim
print "KNN # Neighbors: ", best_knn_k
print "Resulting R^2:", best_r2
            

PCA Dimensions:  5 	KNN # Neighbors:  20 	R^2:  0.185045419165
PCA Dimensions:  5 	KNN # Neighbors:  22 	R^2:  0.186554709082
PCA Dimensions:  5 	KNN # Neighbors:  24 	R^2:  0.165267005097
PCA Dimensions:  5 	KNN # Neighbors:  26 	R^2:  0.155425048003
PCA Dimensions:  5 	KNN # Neighbors:  28 	R^2:  0.189712643067
PCA Dimensions:  5 	KNN # Neighbors:  30 	R^2:  0.172239011839
PCA Dimensions:  5 	KNN # Neighbors:  32 	R^2:  0.159805002595
PCA Dimensions:  5 	KNN # Neighbors:  34 	R^2:  0.183440761743
PCA Dimensions:  5 	KNN # Neighbors:  36 	R^2:  0.191036919633
PCA Dimensions:  5 	KNN # Neighbors:  38 	R^2:  0.169569441049
PCA Dimensions:  5 	KNN # Neighbors:  40 	R^2:  0.180843791249
PCA Dimensions:  5 	KNN # Neighbors:  42 	R^2:  0.188372406874
PCA Dimensions:  5 	KNN # Neighbors:  44 	R^2:  0.211969646583
PCA Dimensions:  5 	KNN # Neighbors:  46 	R^2:  0.192958580787
PCA Dimensions:  5 	KNN # Neighbors:  48 	R^2:  0.193462705643
PCA Dimensions:  5 	KNN # Neighbors:  50 	R^2:  0.19288

Final:

In [28]:
pca = PCA(n_components=12)
pca.fit(x_train)
x_train_reduced = pca.transform(x_train)
x_test_reduced = pca.transform(x_test)


final_knn_model = KNN(n_neighbors = 30)
final_knn_model.fit(x_train_reduced, y_train.values)

final_knn_model.score(x_test_reduced, y_test.values)

0.2613620664877182